# Debug Tlaib Income Security Package Reforms

Testing specific household scenarios to identify issues with reform impacts.

In [1]:
import pandas as pd
from policyengine_us import Simulation

pd.set_option('display.float_format', '{:.2f}'.format)

c:\Users\dtsax\envs\pe\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Helper Functions

In [2]:
def create_situation_baby_bonus():
    """Single parent with newborn (under 1), $50,000 income"""
    return {
        "people": {
            "parent": {
                "age": {"2026": 30},
                "employment_income": {"2026": 50_000},
            },
            "baby": {
                "age": {"2026": 0},
            },
        },
        "families": {"family": {"members": ["parent", "baby"]}},
        "marital_units": {"marital_unit": {"members": ["parent"]}},
        "tax_units": {"tax_unit": {"members": ["parent", "baby"]}},
        "spm_units": {"spm_unit": {"members": ["parent", "baby"]}},
        "households": {"household": {"members": ["parent", "baby"]}},
    }

def create_situation_ecpa():
    """Married, three kids (6, 18, 19), $50,000 income"""
    return {
        "people": {
            "adult1": {
                "age": {"2026": 40},
                "employment_income": {"2026": 50_000},
            },
            "adult2": {
                "age": {"2026": 38},
            },
            "child1": {
                "age": {"2026": 6},
            },
            "child2": {
                "age": {"2026": 18},
            },
            "child3": {
                "age": {"2026": 19},
            },
        },
        "families": {"family": {"members": ["adult1", "adult2", "child1", "child2", "child3"]}},
        "marital_units": {"marital_unit": {"members": ["adult1", "adult2"]}},
        "tax_units": {"tax_unit": {"members": ["adult1", "adult2", "child1", "child2", "child3"]}},
        "spm_units": {"spm_unit": {"members": ["adult1", "adult2", "child1", "child2", "child3"]}},
        "households": {"household": {"members": ["adult1", "adult2", "child1", "child2", "child3"]}},
    }

def create_situation_boost():
    """Single person, $100,000 income"""
    return {
        "people": {
            "adult": {
                "age": {"2026": 35},
                "employment_income": {"2026": 100_000},
            },
        },
        "families": {"family": {"members": ["adult"]}},
        "marital_units": {"marital_unit": {"members": ["adult"]}},
        "tax_units": {"tax_unit": {"members": ["adult"]}},
        "spm_units": {"spm_unit": {"members": ["adult"]}},
        "households": {"household": {"members": ["adult"]}},
    }

def create_situation_combined():
    """Married, three kids (newborn, 18, 19), $100,000 income"""
    return {
        "people": {
            "adult1": {
                "age": {"2026": 40},
                "employment_income": {"2026": 100_000},
            },
            "adult2": {
                "age": {"2026": 38},
            },
            "baby": {
                "age": {"2026": 0},
            },
            "child2": {
                "age": {"2026": 18},
            },
            "child3": {
                "age": {"2026": 19},
            },
        },
        "families": {"family": {"members": ["adult1", "adult2", "baby", "child2", "child3"]}},
        "marital_units": {"marital_unit": {"members": ["adult1", "adult2"]}},
        "tax_units": {"tax_unit": {"members": ["adult1", "adult2", "baby", "child2", "child3"]}},
        "spm_units": {"spm_unit": {"members": ["adult1", "adult2", "baby", "child2", "child3"]}},
        "households": {"household": {"members": ["adult1", "adult2", "baby", "child2", "child3"]}},
    }

def test_scenario(situation, name, reform_params=None):
    """Test a scenario with baseline and reform"""
    print(f"\n{'='*60}")
    print(f"Testing: {name}")
    print(f"{'='*60}")
    
    # Baseline
    baseline = Simulation(situation=situation)
    
    # Reform
    if reform_params is None:
        reform_params = {}
    
    reformed = Simulation(
        situation=situation,
        reform=reform_params
    )
    
    # Standard variables that exist in both baseline and reform
    standard_variables = [
        # Income components
        "employment_income",
        "adjusted_gross_income",
        
        # Standard tax credits (check if affected)
        "income_tax_before_refundable_credits",
        "income_tax_non_refundable_credits",
        "non_refundable_ctc",
        "income_tax_refundable_credits",
        "refundable_ctc",
        "eitc",
        "cdcc",
        "income_tax",
        
        # Other benefit programs (check for unintended changes)
        "snap",
        "wic",
        "tanf",
        "social_security",
        "ssi",
        "unemployment_compensation",
        "free_school_meals",
        "reduced_price_school_meals",
        
        # Aggregated outcomes
        "household_benefits",
        "household_tax_before_refundable_credits",
        "household_refundable_tax_credits",
        "household_net_income",
    ]
    
    # Reform-specific variables (only exist in reform)
    reform_variables = [
        "baby_bonus_act_payment",
        "boost_act_payment",
        "boost_act_tax",
        "ecpa_child_benefit",
        "ecpa_adult_dependent_credit",
        "ecpa_filer_credit",
    ]
    
    results = []
    
    # Calculate standard variables
    for var in standard_variables:
        try:
            base_val = baseline.calculate(var, "2026")[0]
            reform_val = reformed.calculate(var, "2026")[0]
            diff = reform_val - base_val
            
            results.append({
                "Variable": var,
                "Baseline": base_val,
                "Reform": reform_val,
                "Change": diff
            })
        except Exception as e:
            results.append({
                "Variable": var,
                "Baseline": "ERROR",
                "Reform": "ERROR",
                "Change": str(e)[:50]
            })
    
    # Calculate reform-specific variables (baseline = 0)
    for var in reform_variables:
        try:
            reform_val = reformed.calculate(var, "2026")[0]
            results.append({
                "Variable": var,
                "Baseline": 0,
                "Reform": reform_val,
                "Change": reform_val
            })
        except Exception as e:
            # Variable doesn't exist in this reform configuration
            pass
    
    df = pd.DataFrame(results)
    
    # Only show rows with non-zero baseline or reform values
    df_nonzero = df[
        ((df["Baseline"] != 0) & (df["Baseline"] != "ERROR")) | 
        ((df["Reform"] != 0) & (df["Reform"] != "ERROR"))
    ]
    
    print("\n=== NON-ZERO VALUES ===")
    print(df_nonzero.to_string(index=False))
    
    # Show rows with changes
    df_changes = df[
        (df["Change"] != 0) & (df["Change"] != "ERROR")
    ]
    print("\n=== VALUES THAT CHANGED ===")
    print(df_changes.to_string(index=False))
    
    return df

## Test 1: Baby Bonus (Single Parent with Newborn, $50k)

In [3]:
baby_bonus_reform = {
    "gov.contrib.congress.tlaib.income_security_package.baby_bonus_act.in_effect": {
        "2026-01-01.2100-12-31": True
    }
}

df_baby = test_scenario(
    create_situation_baby_bonus(),
    "Baby Bonus: Single parent with newborn, $50k",
    baby_bonus_reform
)


Testing: Baby Bonus: Single parent with newborn, $50k

=== NON-ZERO VALUES ===
                               Variable  Baseline   Reform  Change
                      employment_income  50000.00 50000.00    0.00
                  adjusted_gross_income  50000.00 50000.00    0.00
   income_tax_before_refundable_credits    548.00   548.00    0.00
      income_tax_non_refundable_credits   2200.00  2200.00    0.00
                     non_refundable_ctc   2200.00  2200.00    0.00
          income_tax_refundable_credits    246.63   246.63    0.00
                                   eitc    246.63   246.63    0.00
                             income_tax    301.37   301.37    0.00
                     household_benefits   2186.02  4186.02 2000.00
household_tax_before_refundable_credits   4373.00  4373.00    0.00
       household_refundable_tax_credits    246.63   246.63    0.00
                   household_net_income  48059.65 50059.65 2000.00

=== VALUES THAT CHANGED ===
            Variable

## Test 2: ECPA (Married, 3 Kids Ages 6, 18, 19, $50k)

In [4]:
ecpa_reform = {
    "gov.contrib.congress.tlaib.income_security_package.end_child_poverty_act.in_effect": {
        "2026-01-01.2100-12-31": True
    }
}

df_ecpa = test_scenario(
    create_situation_ecpa(),
    "ECPA: Married, 3 kids (6, 18, 19), $50k",
    ecpa_reform
)


Testing: ECPA: Married, 3 kids (6, 18, 19), $50k

=== NON-ZERO VALUES ===
                               Variable  Baseline   Reform   Change
                      employment_income  50000.00 50000.00     0.00
                  adjusted_gross_income  50000.00 50000.00     0.00
   income_tax_before_refundable_credits      0.00  1780.00  1780.00
      income_tax_non_refundable_credits   1780.00     0.00 -1780.00
                     non_refundable_ctc   1780.00  1780.00     0.00
          income_tax_refundable_credits   4755.66  1600.00 -3155.66
                         refundable_ctc   1420.00  1420.00     0.00
                                   eitc   3335.66  3335.66     0.00
                             income_tax  -4755.66   180.00  4935.66
                                   snap   3221.75  3221.75     0.00
                      free_school_meals   1130.96  1130.96     0.00
                     household_benefits   4352.72 15612.72 11260.00
household_tax_before_refundable_credits  

## Test 3: BOOST (Single Person, $100k)

In [5]:
boost_reform = {
    "gov.contrib.congress.tlaib.income_security_package.boost_act.in_effect": {
        "2026-01-01.2100-12-31": True
    }
}

df_boost = test_scenario(
    create_situation_boost(),
    "BOOST: Single person, $100k",
    boost_reform
)


Testing: BOOST: Single person, $100k

=== NON-ZERO VALUES ===
                               Variable  Baseline    Reform  Change
                      employment_income 100000.00 100000.00    0.00
                  adjusted_gross_income 100000.00 100000.00    0.00
   income_tax_before_refundable_credits  13170.00  13170.00    0.00
                             income_tax  13170.00  14920.00 1750.00
                     household_benefits      0.00   3000.00 3000.00
household_tax_before_refundable_credits  25810.07  27560.07 1750.00
                   household_net_income  74189.94  75439.94 1250.00
                      boost_act_payment      0.00   3000.00 3000.00
                          boost_act_tax      0.00   1750.00 1750.00

=== VALUES THAT CHANGED ===
                               Variable  Baseline   Reform  Change
                             income_tax  13170.00 14920.00 1750.00
                     household_benefits      0.00  3000.00 3000.00
household_tax_before_refund

## Test 4: Combined (Married, 3 Kids including Newborn, $100k)

In [6]:
combined_reform = {
    "gov.contrib.congress.tlaib.income_security_package.baby_bonus_act.in_effect": {
        "2026-01-01.2100-12-31": True
    },
    "gov.contrib.congress.tlaib.income_security_package.boost_act.in_effect": {
        "2026-01-01.2100-12-31": True
    },
    "gov.contrib.congress.tlaib.income_security_package.end_child_poverty_act.in_effect": {
        "2026-01-01.2100-12-31": True
    }
}

df_combined = test_scenario(
    create_situation_combined(),
    "Combined: Married, 3 kids (newborn, 18, 19), $100k",
    combined_reform
)


Testing: Combined: Married, 3 kids (newborn, 18, 19), $100k

=== NON-ZERO VALUES ===
                               Variable  Baseline    Reform   Change
                      employment_income 100000.00 100000.00     0.00
                  adjusted_gross_income 100000.00 100000.00     0.00
   income_tax_before_refundable_credits   4440.00   7640.00  3200.00
      income_tax_non_refundable_credits   3200.00      0.00 -3200.00
                     non_refundable_ctc   3200.00   3200.00     0.00
          income_tax_refundable_credits      0.00    700.00   700.00
                             income_tax   4440.00   7940.00  3500.00
                     household_benefits   2186.02  24446.02 22260.00
household_tax_before_refundable_credits  12663.31  16863.31  4200.00
       household_refundable_tax_credits      0.00    700.00   700.00
                   household_net_income  89522.70 108282.70 18760.00
                      boost_act_payment      0.00   3000.00  3000.00
                 

## Detailed Person-Level Analysis

Let's check person-level values to see if payments are being assigned correctly:

In [7]:
def analyze_person_level(situation, name, reform_params):
    print(f"\n{'='*60}")
    print(f"Person-level analysis: {name}")
    print(f"{'='*60}")
    
    sim = Simulation(situation=situation, reform=reform_params)
    
    # Get all person IDs
    people = list(situation["people"].keys())
    
    # Standard variables
    standard_vars = [
        "age",
        "employment_income",
        "is_tax_unit_dependent",
    ]
    
    # Reform-specific variables
    reform_vars = [
        "baby_bonus_act_payment",
        "boost_act_payment",
        "ecpa_child_benefit",
        "ecpa_adult_dependent_credit",
    ]
    
    results = []
    for person_idx, person in enumerate(people):
        row = {"Person": person}
        
        # Calculate standard variables
        for var in standard_vars:
            try:
                vals = sim.calculate(var, "2026")
                row[var] = vals[person_idx]
            except Exception as e:
                row[var] = f"ERROR: {str(e)[:30]}"
        
        # Calculate reform-specific variables
        for var in reform_vars:
            try:
                vals = sim.calculate(var, "2026")
                row[var] = vals[person_idx]
            except Exception as e:
                # Variable doesn't exist in this reform
                row[var] = 0
        
        results.append(row)
    
    df = pd.DataFrame(results)
    print(df.to_string(index=False))
    return df

In [8]:
# Baby Bonus person-level
analyze_person_level(
    create_situation_baby_bonus(),
    "Baby Bonus",
    baby_bonus_reform
)


Person-level analysis: Baby Bonus
Person   age  employment_income  is_tax_unit_dependent  baby_bonus_act_payment  boost_act_payment  ecpa_child_benefit  ecpa_adult_dependent_credit
parent 30.00           50000.00                  False                    0.00               0.00                0.00                         0.00
  baby  0.00               0.00                   True                 2000.00               0.00                0.00                         0.00


,Person,age,employment_income,is_tax_unit_dependent,baby_bonus_act_payment,boost_act_payment,ecpa_child_benefit,ecpa_adult_dependent_credit
0,parent,30.00,50000.00,False,0.00,0.00,0.00,0.00
1,baby,0.00,0.00,True,2000.00,0.00,0.00,0.00


In [9]:
# ECPA person-level
analyze_person_level(
    create_situation_ecpa(),
    "ECPA",
    ecpa_reform
)


Person-level analysis: ECPA
Person   age  employment_income  is_tax_unit_dependent  baby_bonus_act_payment  boost_act_payment  ecpa_child_benefit  ecpa_adult_dependent_credit
adult1 40.00           50000.00                  False                    0.00               0.00                0.00                         0.00
adult2 38.00               0.00                  False                    0.00               0.00                0.00                         0.00
child1  6.00               0.00                   True                    0.00               0.00             5630.00                         0.00
child2 18.00               0.00                   True                    0.00               0.00             5630.00                         0.00
child3 19.00               0.00                   True                    0.00               0.00                0.00                       700.00


,Person,age,employment_income,is_tax_unit_dependent,baby_bonus_act_payment,boost_act_payment,ecpa_child_benefit,ecpa_adult_dependent_credit
0,adult1,40.00,50000.00,False,0.00,0.00,0.00,0.00
1,adult2,38.00,0.00,False,0.00,0.00,0.00,0.00
2,child1,6.00,0.00,True,0.00,0.00,5630.00,0.00
3,child2,18.00,0.00,True,0.00,0.00,5630.00,0.00
4,child3,19.00,0.00,True,0.00,0.00,0.00,700.00


In [10]:
# BOOST person-level
analyze_person_level(
    create_situation_boost(),
    "BOOST",
    boost_reform
)


Person-level analysis: BOOST
Person   age  employment_income  is_tax_unit_dependent  baby_bonus_act_payment  boost_act_payment  ecpa_child_benefit  ecpa_adult_dependent_credit
 adult 35.00          100000.00                  False                    0.00            3000.00                0.00                         0.00


,Person,age,employment_income,is_tax_unit_dependent,baby_bonus_act_payment,boost_act_payment,ecpa_child_benefit,ecpa_adult_dependent_credit
0,adult,35.00,100000.00,False,0.00,3000.00,0.00,0.00


In [11]:
# Combined person-level
analyze_person_level(
    create_situation_combined(),
    "Combined",
    combined_reform
)


Person-level analysis: Combined
Person   age  employment_income  is_tax_unit_dependent  baby_bonus_act_payment  boost_act_payment  ecpa_child_benefit  ecpa_adult_dependent_credit
adult1 40.00          100000.00                  False                    0.00            3000.00                0.00                         0.00
adult2 38.00               0.00                  False                    0.00            3000.00                0.00                         0.00
  baby  0.00               0.00                   True                 2000.00               0.00             5630.00                         0.00
child2 18.00               0.00                   True                    0.00               0.00             5630.00                         0.00
child3 19.00               0.00                   True                    0.00            3000.00                0.00                       700.00


,Person,age,employment_income,is_tax_unit_dependent,baby_bonus_act_payment,boost_act_payment,ecpa_child_benefit,ecpa_adult_dependent_credit
0,adult1,40.00,100000.00,False,0.00,3000.00,0.00,0.00
1,adult2,38.00,0.00,False,0.00,3000.00,0.00,0.00
2,baby,0.00,0.00,True,2000.00,0.00,5630.00,0.00
3,child2,18.00,0.00,True,0.00,0.00,5630.00,0.00
4,child3,19.00,0.00,True,0.00,3000.00,0.00,700.00


## Summary Comparison

Net income impact for each scenario:

In [12]:
summary_data = []

for df, name in [
    (df_baby, "Baby Bonus"),
    (df_ecpa, "ECPA"),
    (df_boost, "BOOST"),
    (df_combined, "Combined")
]:
    net_income_row = df[df["Variable"] == "household_net_income"]
    if not net_income_row.empty:
        summary_data.append({
            "Scenario": name,
            "Baseline Net Income": net_income_row["Baseline"].values[0],
            "Reform Net Income": net_income_row["Reform"].values[0],
            "Change": net_income_row["Change"].values[0]
        })

summary_df = pd.DataFrame(summary_data)
print("\n" + "="*60)
print("SUMMARY: Net Income Impact")
print("="*60)
print(summary_df.to_string(index=False))


SUMMARY: Net Income Impact
  Scenario  Baseline Net Income  Reform Net Income   Change
Baby Bonus             48059.65           50059.65  2000.00
      ECPA             55283.38           61607.72  6324.34
     BOOST             74189.94           75439.94  1250.00
  Combined             89522.70          108282.70 18760.00
